In [1]:
#make sure u have statsmodels version 0.12.2, or else u will be given boxcox error. for the rest of deps, earliest versions as of 2022 work

import pandas as pd
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt 

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import scipy

import seaborn as sns
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tools.eval_measures import rmse
from scipy import signal
from scipy.stats import shapiro
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.preprocessing import StandardScaler
from scipy.stats import jarque_bera as jb
from itertools import combinations
from statsmodels.stats.diagnostic import acorr_ljungbox as ljung
import itertools

from pandas.plotting import autocorrelation_plot

plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (12, 10) 
plt.rcParams['axes.grid']=True
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']

import requests

from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [6]:
def forecaster(lat, long, horizon):
    #for lat, pass the latitude of forecasted location 
    #for long, pass the longitude of forecasted location 
    #for horizon, pass the amount of 5 day intervals you would like to forecast. 
    #   example. horizon=5 will give a 25 day forecast 
    ### IMPORTANT: REMEMBER THIS GIVES YOU THE HORIZON FROM THE LAST DATE IN THE HISTORIC DATASET.
    ### FOR EXAMPLE, CURRENTLY 2022-02-28 IS THE MOST RECENT DATE IN ERA5 2M LAND TEMP. THIS MEANS THAT 
    ### PASSING horizon=1 WILL GIVE A FORECAST FOR MARCH 1-5
    
    #fetch radiation data
    dataset = 'era5_land_surface_solar_radiation_downwards-hourly'
    my_token = 'REDACTED'
    my_url = 'https://api.dclimate.net/apiv3/grid-history/' + dataset + '/' + str(lat) + '_' + str(long)
    head = {"Authorization": my_token}
    r = requests.get(my_url, headers=head)
    data = r.json()["data"]
    index = pd.to_datetime(list(data.keys()))
    values = [float(s.split()[0]) if s else None for s in data.values()]
    series = pd.Series(values, index=index)
    df_solar = series.to_frame(name='Value')
    
    #fetch temperature data
    dataset = 'era5_land_2m_temp-hourly'
    my_url = 'https://api.dclimate.net/apiv3/grid-history/' + dataset + '/' + str(lat) + '_' + str(long)
    r = requests.get(my_url, headers=head)
    data = r.json()["data"]
    index = pd.to_datetime(list(data.keys()))
    values = [float(s.split()[0]) if s else None for s in data.values()]
    series = pd.Series(values, index=index)
    df_temp = series.to_frame(name='Value')
    
    #transform data    
    df_solar.index = pd.to_datetime(df_solar.index, utc=True)
    df_solar = df_solar[~df_solar.index.astype(str).str.contains('02-29')]
    bucketed = df_solar.resample('D').Value.sum()
    '''
        if df in [df_temp]:
            df.index = pd.to_datetime(df.index, utc=True)
            df = df[~df.index.astype(str).str.contains('02-29')]
            bucketed = df.resample('D').Value.mean()
    '''
    bucketed = bucketed.to_frame()
    df_solar = bucketed
    fivebucketed = df_solar.resample('5D').Value.mean()
    fivebucketed = fivebucketed.to_frame()
    df_solar = fivebucketed
    df_solar.index = pd.to_datetime(df_solar.index, utc=True)
    train_bcox, bcox_lam = boxcox(df_solar["Value"])
    train_log = np.log(df_solar["Value"])
    hw_model = ExponentialSmoothing(df_solar["Value"],
                              trend    = None,
                              seasonal = "add", 
                              seasonal_periods=73, 
                              damped=False).fit(use_boxcox='log')

    hw_fitted = hw_model.fittedvalues

    hw_resid = hw_model.resid

        #Adding the mean of the residuals to correct the bias.
    py_hw = hw_model.forecast(horizon) + np.mean(hw_resid)
    
    results = {}
    results['solar'] = py_hw.values
    
    df_temp.index = pd.to_datetime(df_temp.index, utc=True)
    df_temp = df_temp[~df_temp.index.astype(str).str.contains('02-29')]
    bucketed = df_temp.resample('D').Value.mean()
    bucketed = bucketed.to_frame()
    df_temp = bucketed
    fivebucketed = df_temp.resample('5D').Value.mean()
    fivebucketed = fivebucketed.to_frame()
    df_temp = fivebucketed
    df_temp.index = pd.to_datetime(df_temp.index, utc=True)
    train_bcox, bcox_lam = boxcox(df_temp["Value"])
    train_log = np.log(df_temp["Value"])
    hw_model = ExponentialSmoothing(df_temp["Value"],
                              trend    = None,
                              seasonal = "add", 
                              seasonal_periods=73, 
                              damped=False).fit(use_boxcox='log')

    hw_fitted = hw_model.fittedvalues

    hw_resid = hw_model.resid

        #Adding the mean of the residuals to correct the bias.
    py_hw = hw_model.forecast(horizon) + np.mean(hw_resid)

    results['temp'] = py_hw.values
    
    
    return results


In [9]:
#sample forecast generated 365 days in the future for 40 -120 
results = forecaster(40, -120, 73)

/tmp/ipykernel_47854/830075862.py:56: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead
  hw_model = ExponentialSmoothing(df_solar["Value"],
/tmp/ipykernel_47854/830075862.py:95: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead
  hw_model = ExponentialSmoothing(df_temp["Value"],


In [10]:
print(results['temp'])
print(results['solar'])

[33.46983798 35.21471876 37.22034897 37.00303574 38.04566372 39.6565721
 41.01056108 42.34796831 42.1474649  44.91200689 47.06922182 48.59066523
 50.76116876 51.8200018  53.88506846 53.90079157 54.29462093 57.67795274
 59.69595335 60.25189757 63.30364088 65.12571432 66.27839171 68.30204992
 70.65319206 70.40895146 70.734782   70.97835954 71.78229749 72.13966923
 70.37801052 69.27333914 68.92246779 68.81974798 67.74141401 67.39837029
 65.09010656 63.49000505 61.42296134 59.82096094 57.72219904 57.87757799
 53.11936001 51.43583863 50.24361039 49.21272131 46.70235686 43.68849481
 41.6928037  39.26269313 36.59098546 36.23935578 33.27067764 32.25688838
 31.11782063 28.8114524  28.20348399 24.54443394 25.3582842  26.5771656
 25.42212605 27.27034724 27.59387527 27.54196249 28.04797079 27.48864607
 27.47853451 29.71410197 29.87977832 31.52497145 30.98694213 31.24018286
 32.50243185]
[5315.6078346  5641.79583679 5802.91396422 6074.74483393 6580.73151357
 6730.09033507 6969.89045232 6941.6482444

These results are the results for 1 year of predictions with 5 days between each prediction for 40,-120 starting on february 28th (the last day of data)